In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#carregando os datasets que serão utilizados na análise
df_orders = pd.read_csv('Datasets/olist_orders_dataset.csv')
df_order_items = pd.read_csv('Datasets/olist_order_items_dataset.csv')
df_order_reviews = pd.read_csv('Datasets/olist_order_reviews_dataset.csv')

In [20]:
#unindo as tabelas
df_completo = pd.merge(df_orders, df_order_items, on='order_id', how='inner')
df_completo = pd.merge(df_completo, df_order_reviews, on='order_id', how='inner')

In [25]:
#colunas de data para conversão
cols_data = [
    'order_purchase_timestamp', 
    'order_estimated_delivery_date', 
    'order_delivered_customer_date'
]

#convertendo para datetime
for col in cols_data:
    df_completo[col] = pd.to_datetime(df_completo[col])

In [26]:
#realizando os cálculos de métricas logísticas
df_completo['dias_atraso'] = (
    df_completo['order_delivered_customer_date'] - 
    df_completo['order_estimated_delivery_date']
).dt.days

#tempo total de espera do cliente
df_completo['tempo_espera'] = (
    df_completo['order_delivered_customer_date'] - 
    df_completo['order_purchase_timestamp']
).dt.days

#status em binário 
df_completo['is_late'] = df_completo['dias_atraso'].apply(lambda x: 1 if x > 0 else 0)

In [29]:
#diagnóstico de Outliers (dias_atraso)
#variável temporária sem nulos para o cálculo estatístico
delay_clean = df_completo['dias_atraso'].dropna()

Q1 = delay_clean.quantile(0.25)
Q3 = delay_clean.quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

outliers_sup = delay_clean[delay_clean > limite_superior].count()
outliers_inf = delay_clean[delay_clean < limite_inferior].count()

print("\n--- Diagnóstico de Outliers (Estatístico) ---")
print(f"Valor Mínimo encontrado: {delay_clean.min()} dias")
print(f"Valor Máximo encontrado: {delay_clean.max()} dias")
print(f"Limite Superior Sugerido (IQR): {limite_superior:.2f} dias")
print(f"Quantidade de Outliers Superiores: {outliers_sup}")
print(f"Quantidade de Outliers Inferiores: {outliers_inf}")


--- Diagnóstico de Outliers (Estatístico) ---
Valor Mínimo encontrado: -147 dias
Valor Máximo encontrado: 188 dias
Limite Superior Sugerido (IQR): 8.00 dias
Quantidade de Outliers Superiores: 2697
Quantidade de Outliers Inferiores: 2161


In [30]:
#removendo pedidos sem data de entrega
df_clean = df_completo.dropna(subset=['order_delivered_customer_date']).copy()

#aplicando filtro de negócio (removendo ruídos extremos)
#matendo: entregas até 40 dias antes do prazo e até 90 dias após o prazo
df_clean = df_clean[
    (df_clean['dias_atraso'] >= -40) & 
    (df_clean['dias_atraso'] <= 90)
]

#verificando o novo volume de dados
print(f"Volume original: {len(df_completo)}")
print(f"Volume após limpeza: {len(df_clean)}")
print(f"Porcentagem mantida: {(len(df_clean)/len(df_completo))*100:.2f}%")

Volume original: 110012
Volume após limpeza: 109290
Porcentagem mantida: 99.34%
